In [6]:
import casadi as ca
import numpy as np
import time


inf


In [4]:
def shift_movement(T, t0, x0, u, f):
	# 小车运动到下一个位置
    f_value = f(x0, u[:, 0])
    st = x0 + T*f_value
    # 时间增加
    t = t0 + T
    # 准备下一个估计的最优控制，因为u[:, 0]已经采纳，我们就简单地把后面的结果提前
    u_end = ca.horzcat(u[:, 1:], u[:, -1])
    return t, st, u_end.T

In [3]:
T= 0.2
N= 10
rob_size= 0.3
v_max= 0.6
omega_max= np.pi/4

# states
x= ca.SX.sym('x')
y= ca.SX.sym('y')
theta= ca.SX.sym('theta')
states= ca.vertcat(x, y, theta)
num_states= states.size()[0]

# control inputs
v= ca.SX.sym('v')
omega= ca.SX.sym('omega')
u= ca.vertcat(v, omega)
num_u= u.size()[0]

#state equations
dx= v* ca.cos(theta)
dy= v* ca.sin(theta)
dtheta= omega

dstates= ca.vertcat(dx,dy,dtheta)

f= ca.Function('f', [states,u], [dstates], ['states', 'u'], ['dstates'])

In [16]:
U= ca.SX.sym('U', num_u, N)
X= ca.SX.sym('X', num_states, N+1)
P= ca.SX.sym('P', num_states+num_states)

X[:, 0]= P[: 3]

for i in range(N):
    f_now= f(X[:,i], U[:,i])
    X[:, i+1]= f_now+ X[:, i]

ff= ca.Function('ff', [U, P], [X], ['input_U', 'target_P'], ['states_X'])

Q = np.array([[1.0, 0.0, 0.0],[0.0, 5.0, 0.0],[0.0, 0.0, .1]])
R = np.array([[0.5, 0.0], [0.0, 0.05]])

obj= 0
for i in range (N):
    obj+= ca.mtimes([(X[:,i]- P[3:]).T, Q, X[:,i]- P[3:]])+ ca.mtimes([U[:, i].T, R, U[:, i]])

g=[]
for i in range (N+1):
    g.append(X[0, i])
    g.append(X[1, i])

nlp_prob = {'f': obj, 'x': ca.reshape(U, -1, 1), 'p':P, 'g':ca.vertcat(*g)}

opts_setting = {'ipopt.max_iter':100,
                'ipopt.print_level':0,
                'print_time':0,
                'ipopt.acceptable_tol':1e-8,
                'ipopt.acceptable_obj_change_tol':1e-6}

solver = ca.nlpsol('solver', 'ipopt', nlp_prob, opts_setting)

In [20]:
lb_g= -2.0
ub_g= 2.0

lb_u=[]
ub_u=[]

for _ in range (N):
    lb_u.append(-v_max)
    lb_u.append(-omega_max)
    ub_u.append(v_max)
    ub_u.append(omega_max)

t0= 0.0
x0= np.array([0.0, 0.0, 0.0]).reshape(-1, 1)
u0= np.array([0.0, 0.0]*N).reshape(-1, 2)
xs= np.array([1.5, 1.5, 0.0]).reshape(-1, 1)

x_c = [] # 存储系统的状态
u_c = [] # 存储控制全部计算后的控制指令
t_c = [] # 保存时间
xx = [] # 存储每一步机器人位置
sim_time = 20.0 # 仿真时长
index_t = [] # 存储时间戳，以便计算每一步求解的时间

## 开始仿真
mpciter = 0 # 迭代计数器
start_time = time.time() # 获取开始仿真时间

### 终止条件为小车和目标的欧式距离小于0.01或者仿真超时
while(np.linalg.norm(x0-xs)>1e-2 and mpciter-sim_time/T<0.0 ):
    ### 初始化优化参数
    c_p = np.concatenate((x0, xs))
    ### 初始化优化目标变量
    init_control = ca.reshape(u0, -1, 1)
    ### 计算结果并且
    t_ = time.time()
    res = solver(x0=init_control, p=c_p, lbg=lb_g, lbx=lb_u, ubg=ub_g, ubx=ub_u)
    index_t.append(time.time()- t_)
    ### 获得最优控制结果u
    u_sol = ca.reshape(res['x'], num_u, N) # 记住将其恢复U的形状定义
    ###
    ff_value = ff(u_sol, c_p) # 利用之前定义ff函数获得根据优化后的结果
        						  # 小车之后N+1步后的状态（n_states, N+1）
    ### 存储结果
    x_c.append(ff_value)
    u_c.append(u_sol[:, 0])
    t_c.append(t0)

    ### 根据数学模型和MPC计算的结果移动小车并且准备好下一个循环的初始化目标
    t0, x0, u0 = shift_movement(T, t0, x0, u_sol, f)

	### 存储小车的位置
    x0 = ca.reshape(x0, -1, 1)
    xx.append(x0.full())
    ### 计数器+1
    mpciter = mpciter + 1






******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



NameError: name 'shift_movement' is not defined